# Effect of UBI on labor supply in OG-USA small open economy

Testing the income effect.

Goals:

* What's the implied income effect when people get a \$1,000-per-person-per-year UBI, in a small open economy?
* How does this vary with frisch and epsilon?
* How does this compare to microeconometric studies on the income effect?
* How does it compare to other models like CBO and PWBM?
* How does it vary with lifetime income level and age?

Intermediate questions:
* What is the range of the implied UBI based on its ETR effect? Is it roughly the `taxcalc`-supplied UBI value multiplied by the number of people per tax unit?
* What's the overall labor response to the \$1,000 UBI across lifetime income groups and age?

## Setup

In [1]:
import pandas as pd
import numpy as np
import ogusa
import os
import microdf as mdf

In [2]:
PATH = 'default'
SS_VARS_FILE = 'SS/SS_vars.pkl'
base = pd.read_pickle(os.path.join(PATH, 'OUTPUT_BASELINE', SS_VARS_FILE))
base_params = pd.read_pickle(os.path.join(PATH, 'OUTPUT_BASELINE/model_params.pkl'))
reform = pd.read_pickle(os.path.join(PATH, 'OUTPUT_REFORM', SS_VARS_FILE))
reform_params = pd.read_pickle(os.path.join(PATH, 'OUTPUT_REFORM/model_params.pkl'))

## Utilities

In [3]:
# Map each skill level to a printed value and its total population weight.
# Un-nest lambdas.
lambdas = [i[0] for i in base_params.lambdas.tolist()]
# Create DataFrame.
LIFETIME_INCOME_BUCKETS = ['0-25%', '25-50%', '50-70%', '70-80%', '80-90%',
                           '90-99%', 'Top 1%']
lifetime_income_map = pd.DataFrame({'skill': np.arange(7),
                                    'lifetime_income_group': LIFETIME_INCOME_BUCKETS,
                                    'lifetime_income_pop_share': lambdas})

In [4]:
# Map each age to its population weight.
omegas_ss = reform_params.omega[-1, :]  # Final time period.
age_pop_map = pd.DataFrame({'age': np.arange(21, 101),
                            'age_pop_share': omegas_ss})

In [5]:
def age_skill_df(mats, colnames):
    '''
    Produces DataFrame by age and lifetime income group
    based on 2d numpy arrays for a given value.
    
    Args:
        mats: List of 2d JxS numpy array. Can also be a singleton.
        colnames: List of column names the value should take.
            Can also be a singleton.
        
    Returns:
        DataFrame with columns for age, lifetime_income_group, and
        <colnames>.
    '''
    # Make args lists if they're not already.
    if type(mats) != list:
        mats = [mats]
        colnames = [colnames]
    df_l = []
    num = len(mats)
    assert len(colnames) == num
    for i in range(num):
        # Melt and rename.
        tmp = pd.DataFrame(mats[i]).reset_index().melt('index')
        tmp.columns = ['age', 'skill', colnames[i]]
        tmp.set_index(['age', 'skill'], inplace=True)
        df_l.append(tmp)
    # Merge all DataFrames.
    df = pd.concat(df_l, axis=1).reset_index()
    # Age starts at 21.
    df.age += 21
    # Map lifetime_income_group by skill index.
    df = df.merge(lifetime_income_map, on='skill')
    df = df.merge(age_pop_map, on='age')
    # Population share for sj is the product.
    df['pop_share'] = df.age_pop_share * df.lifetime_income_pop_share
    return df.drop('skill', axis=1)

## Create data

### UBI amounts

As implied through the ETR.

In [6]:
assert np.allclose(base['rss'], reform['rss'])
assert np.allclose(base['wss'], reform['wss'])
assert np.allclose(base['factor_ss'], reform['factor_ss'])

Specifications can come from baseline.

In [7]:
p = ogusa.Specifications(baseline=True, time_path=False)

In [8]:
etr_params_3D = np.tile(
    np.reshape(reform_params.etr_params[-1, :, :],
               (reform_params.S, 1, reform_params.etr_params.shape[2])),
    (1, reform_params.J, 1))

In [9]:
def net_tax(ss_vars, params, p):
    '''
    Calculates net tax amount for each sj, given steady-state variables,
    except for ETR parameters which can diverge.
    
    Args:
        ss_vars: Dict from a ss_vars.pkl.
        params: params object containing etr_params.
        p: ogusa.Specifications object.
        
    Returns:
        sj numpy array representing the net tax.
    '''
    etr_params_3D = np.tile(
        np.reshape(params.etr_params[-1, :, :],
                   (params.S, 1, params.etr_params.shape[2])),
        (1, params.J, 1))
    return ogusa.tax.net_taxes(
        r=ss_vars['rss'],
        w=ss_vars['wss'], 
        b=ss_vars['bssmat_s'],
        n=ss_vars['nssmat'],
        bq=ss_vars['bqssmat'],  # Bequests received.
        factor=ss_vars['factor_ss'],
        tr=ss_vars['TR_ss'],
        theta=ss_vars['theta'],
        t=None,
        j=None,
        shift=False,
        method='SS',
        e=p.e,
        etr_params=etr_params_3D,
        p=p)

In [10]:
# Calculate net tax for the baseline conditions,
# varying only the ETR, which governs the UBI.
net_tax_ubi = net_tax(base, reform_params, p)
net_tax_base = net_tax(base, base_params, p)

In [11]:
ubi = base['factor_ss'] * (net_tax_base - net_tax_ubi)

### Percent change in income

In [12]:
def y_aftertax(ss_vars, params, p):
    # Should this use yss_before_tax_mat instead?
    # What about bequests and government transfers received?
    return (ss_vars['rss'] * ss_vars['bssmat_s'] + 
            ss_vars['wss'] * p.e * ss_vars['nssmat'] -
            net_tax(ss_vars, params, p))

In [13]:
y_aftertax_base = base['factor_ss'] * y_aftertax(base, base_params, p)

### Combine into a `DataFrame`

With one row per sj.

In [14]:
def calc_ratios(df):
    # Calculate percentage change in after-tax income from UBI.
    df['y_diff'] = df.ubi / df.y_aftertax_base
    # Calculate percentage change in labor response and consumption.
    df['n_diff'] = df.n_reform / df.n_base - 1
    df['c_diff'] = df.c_reform / df.c_base - 1
    # Calculate elasticity.
    df['elasticity'] = df.n_diff / df.y_diff
    df['elasticity_display'] = df.elasticity.round(2)
    # Create variables for plotting.
    df['y_diff_pct'] = (df.y_diff * 100).round(2)
    df['n_diff_pct'] = (df.n_diff * 100).round(2)
    df['c_diff_pct'] = (df.c_diff * 100).round(2)
    df['ubi_print'] = df.ubi.round(0)

In [15]:
sj = age_skill_df([y_aftertax_base, ubi, base['nssmat'], reform['nssmat'],
                   base['cssmat'], reform['cssmat']],
                   ['y_aftertax_base', 'ubi', 'n_base', 'n_reform',
                    'c_base', 'c_reform'])
calc_ratios(sj)

### Aggregate by age and lifetime income

Using `pop_share`.

In [16]:
SUM_VARS = ['ubi', 'y_aftertax_base', 'n_base', 'n_reform', 'c_base', 'c_reform']
W_SUFFIX = '_w'
S_W_SUFFIX = '_ws'
J_W_SUFFIX = '_wj'
sum_vars_w = [i + W_SUFFIX for i in SUM_VARS]
sum_vars_s = [i + S_W_SUFFIX for i in SUM_VARS]
sum_vars_j = [i + J_W_SUFFIX for i in SUM_VARS]
mdf.add_weighted_metrics(sj, SUM_VARS, 'pop_share', divisor=1., suffix=W_SUFFIX)
mdf.add_weighted_metrics(sj, SUM_VARS, 'age_pop_share', divisor=1., suffix=S_W_SUFFIX)
mdf.add_weighted_metrics(sj, SUM_VARS, 'lifetime_income_pop_share', divisor=1., suffix=J_W_SUFFIX)
# Aggregate.
s = sj.groupby('age')[sum_vars_j].sum()
j = sj.groupby('lifetime_income_group')[sum_vars_s].sum()
m = pd.DataFrame(sj[sum_vars_w].sum()).transpose()
# Remove _w suffix.
s.columns = [i[:-3] for i in s.columns]
j.columns = [i[:-3] for i in j.columns]
m.columns = [i[:-2] for i in m.columns]
calc_ratios(s)
calc_ratios(j)
calc_ratios(m)

## Export

In [17]:
sj.to_csv('sj.csv', index=False)
s.reset_index().to_csv('s.csv', index=False)
j.reset_index().to_csv('j.csv', index=False)
m.reset_index().to_csv('m.csv', index=False)

## Other exploratory

Change between base and reform to `yss_before_tax` matches change to `nss` in the first period,
diverges after that.

In [18]:
assert np.allclose(reform['yss_before_tax_mat'][0, :] /
                   base['yss_before_tax_mat'][0, :],
                   reform['nssmat'][0, :] /
                   base['nssmat'][0, :])

In [19]:
assert not np.allclose(reform['yss_before_tax_mat'][1:, :] /
                       base['yss_before_tax_mat'][1:, :],
                       reform['nssmat'][1:, :] /
                       base['nssmat'][1:, :])